In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
    "unsloth/gemma-2-2b-bnb-4bit",             # New small Gemma model!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2060. Max memory: 6.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [3]:
import pandas as pd
from datasets import Dataset
df = pd.read_csv("data/vquanda_Q_train.csv", header=0, delimiter='\t')
df

,question,query
0,Which universities are alma mater to Charles P...,SELECT DISTINCT ?uri WHERE { <Charles_Plosser>...
1,Name the basketball player who played for Phoe...,SELECT DISTINCT ?uri WHERE {?uri <team> <Phoen...
2,Count the key people of the Clinton Foundation?,SELECT DISTINCT COUNT(?uri) WHERE { <Clinton_F...
3,What is the purpose of Maharashtra Chess Assoc...,SELECT DISTINCT ?uri WHERE { <Maharashtra_Ches...
4,Tandem Computers is the subsidiary of which co...,SELECT DISTINCT ?uri WHERE {?uri <subsidiary> ...
...,...,...
3995,How many shows are aired on Comedy Central?,SELECT DISTINCT COUNT(?uri) WHERE {?uri <netwo...
3996,What are the software whose programming langua...,SELECT DISTINCT ?uri WHERE { ?x <language> <Mu...
3997,How many owners are there of lines starting at...,SELECT DISTINCT COUNT(?uri) WHERE { ?x <routeS...
3998,What is the draft team of Dale Mitchell (ice h...,SELECT DISTINCT ?uri WHERE { <Dale_Mitchell_(i...


In [4]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['question', 'query'],
    num_rows: 4000
})

In [5]:

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
The user has provided a question. Convert the question in Natural Language to a SPARQL query. each triple in the SPARQL should follow the order of subject predicate object.
### User Question:
{}
### SPARQL:
{}"""
EOS_TOKEN = tokenizer.eos_token # must add!

# Format dataset
def format_prompts(examples):
    inputs  = examples["question"]
    outputs = examples["query"]
    texts = []
    for _input, _output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(_input, _output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [6]:
dataset = dataset.map(format_prompts, batched=True)
dataset

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'query', 'text'],
    num_rows: 4000
})

In [7]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    dataset_text_field="text",
    # data_collator=collator, # Response only gen
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 200,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/4000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [8]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 2060. Max memory = 6.0 GB.
2.57 GB of memory reserved.


In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 200
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss
1,3.110600
2,3.109500
3,3.073600
4,2.765800
5,2.374100
6,2.081500
7,1.623500
8,1.285300
9,1.073400
10,0.961000


In [10]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

578.8085 seconds used for training.
9.65 minutes used for training.
Peak reserved memory = 3.723 GB.
Peak reserved memory for training = 1.153 GB.
Peak reserved memory % of max memory = 62.05 %.
Peak reserved memory for training % of max memory = 19.217 %.


In [15]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        """How many people in the US are both billionares and evil""", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 500)

<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
The user has provided a question. Convert the question in Natural Language to a SPARQL query. each triple in the SPARQL should follow the order of subject predicate object.
### User Question:
How many people in the US are both billionares and evil
### SPARQL:
SELECT DISTINCT COUNT(?uri) WHERE { ?x <netWorth> <1000000000> . ?x <evil> . ?x <type> <Person>}<eos>


In [12]:
# Save model
model.save_pretrained("query_2b_200step") # Local saving
tokenizer.save_pretrained("query_2b_200step")

('query_2b_200step/tokenizer_config.json',
 'query_2b_200step/special_tokens_map.json',
 'query_2b_200step/tokenizer.model',
 'query_2b_200step/added_tokens.json',
 'query_2b_200step/tokenizer.json')

In [20]:
a = tokenizer("SELECT DISTINCT ?uri WHERE { <Zaman_Shah_Durrani> <title> ?uri. <Salahuddin_Rabbani> <nationality> ?uri}")
a

{'input_ids': [2, 12116, 194240, 1654, 3068, 18765, 612, 968, 235382, 5984, 235298, 128396, 235298, 25978, 52867, 235313, 968, 2563, 235313, 1654, 3068, 235265, 968, 101888, 61823, 235298, 35581, 107167, 235313, 968, 157664, 235313, 1654, 3068, 235270], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
for id in a["input_ids"]:
    print(tokenizer.decode(id))

<bos>
SELECT
 DISTINCT
 ?
uri
 WHERE
 {
 <
Z
aman
_
Shah
_
Dur
rani
>
 <
title
>
 ?
uri
.
 <
Salah
uddin
_
Rab
bani
>
 <
nationality
>
 ?
uri
}
